In [1]:
from tensorflow import keras
import tensorflow as tf
from keras import layers
from keras import backend as k
import numpy as np
import pickle as pk
from scipy.stats import trim_mean
from funcs import singleDescend,getModel,multiDescend, getTestError, getDataSplits, \
    corruptData
def startTFSession():
    #tf.compat.v1.disable_eager_execution()
    return tf.compat.v1.InteractiveSession
session = startTFSession()

In [2]:
(trainFeatures,trainLabels),(testFeatures,testLabels) =  keras.datasets.mnist.load_data()
trainLabels = keras.utils.to_categorical(trainLabels,10)
testLabels = keras.utils.to_categorical(testLabels,10)
trainFeatures.astype(np.float32)
testFeatures.astype(np.float32)
trainFeatures = np.reshape(trainFeatures,(len(trainFeatures),28,28,1))/255
testFeatures = np.reshape(testFeatures,(len(testFeatures),28,28,1))/255

In [3]:
centralRes =[]
for i in range(4):
    model = getModel()
    modelSingle,errSingle = singleDescend(model,trainFeatures,trainLabels,testFeatures,testLabels,session,batchMax=300)
    testError = getTestError(modelSingle,testFeatures,testLabels)
    centralRes.append({'training': errSingle, 'testError': testError,'numWorkers': 1})
    print('###next round\n###'.format(i))

with open('data/CentralResults.pkl','wb') as fi:
    pk.dump(centralRes,fi)

batch: 0, error: 0.855 
batch: 100, error: 0.22499999999999998 
batch: 200, error: 0.08499999999999996 
###next round
###
batch: 0, error: 0.8200000000000001 
batch: 100, error: 0.12 
batch: 200, error: 0.09999999999999998 
###next round
###
batch: 0, error: 0.86 
batch: 100, error: 0.15000000000000002 
batch: 200, error: 0.10499999999999998 
###next round
###
batch: 0, error: 0.91 
batch: 100, error: 0.18000000000000005 
batch: 200, error: 0.17500000000000004 
###next round
###


In [3]:
workerRes = []
#each batch on this actually views (nworkers)*(batchSize) examples...
for numWorkers in [5,10,20,25]:
    splitFeatures = np.split(trainFeatures,numWorkers)
    splitLabels = np.split(trainLabels,numWorkers)
    model,err = multiDescend(splitFeatures,splitLabels,session,testFeatures,testLabels,numWorkers=numWorkers,batchMax=300)
    testError = getTestError(model,testFeatures,testLabels)
    workerRes.append({'training':err,'FinaltestError':testError,'numWorkers':numWorkers})
    print('###next round\n###'.format(numWorkers))
with open('data/FederatedResults.pkl','wb') as fi:
    pk.dump(workerRes,fi)

KeyboardInterrupt: 

In [3]:
workerRes = []
#each batch on this actually views (nworkers)*(batchSize) examples...
for numWorkers in [5,10,20,25]:
    splitFeatures = np.split(trainFeatures,numWorkers)
    splitLabels = np.split(trainLabels,numWorkers)
    
    model,err = multiDescend(splitFeatures,splitLabels,session,testFeatures,testLabels,numWorkers=numWorkers,batchMax=300)
    testError = getTestError(model,testFeatures,testLabels)
    workerRes.append({'training':err,'FinaltestError':testError,'numWorkers':numWorkers})
    print('###next round\n###'.format(numWorkers))
with open('data/CorruptedResults.pkl','wb') as fi:
    pk.dump(workerRes,fi)

KeyboardInterrupt: 

In [ ]:
corruptRate = {5:[0], 10: [0,.1], 20: [0,.05,.1,.2], 25:[0,.05,.1,.2]}
methods = ['trimmedMean','median']
workerResB = {}
for numWorkers in [5,10,20,25]:
    workerResB[numWorkers] = {}
    for beta in corruptRate[numWorkers]:
        for m in methods:
            print("Start nWorkers={}, beta={}, method={}".format(numWorkers,beta,m))
            splitFeatures = np.split(trainFeatures,numWorkers)
            splitLabels = np.split(trainLabels,numWorkers)
            if not beta==0:
                splitLabels = corruptData(splitLabels,beta)
            model,err = multiDescend(splitFeatures,splitLabels,session,testFeatures,testLabels,numWorkers=numWorkers,batchMax=300,method=m,beta=beta)
            testError = getTestError(model,testFeatures,testLabels)
            workerResB[numWorkers][(m,int(numWorkers*beta))] = {'training':err,'FinaltestError':testError,'numWorkers':numWorkers}
            #print('Finished nWorkers={}, corruptRate={}, method={}'.format(numWorkers,beta,m))
            print('done')
with open('data/ByzantineResults.pkl','wb') as fi:
    pk.dump(workerResB,fi)

Start nWorkers=5, beta=0, method=trimmedMean
batch: 0, error: 0.875 
batch: 100, error: 0.18500000000000005 
batch: 200, error: 0.125 
done
Start nWorkers=5, beta=0, method=median
batch: 0, error: 0.92 
batch: 100, error: 0.18000000000000005 
batch: 200, error: 0.09499999999999997 
done
Start nWorkers=10, beta=0, method=trimmedMean
batch: 0, error: 0.925 
batch: 100, error: 0.11499999999999999 
batch: 200, error: 0.08999999999999997 
done
Start nWorkers=10, beta=0, method=median
batch: 0, error: 0.865 
batch: 100, error: 0.10499999999999998 
batch: 200, error: 0.08499999999999996 
done
Start nWorkers=10, beta=0.1, method=trimmedMean
batch: 0, error: 0.905 
batch: 100, error: 0.09999999999999998 
batch: 200, error: 0.07999999999999996 
done
Start nWorkers=10, beta=0.1, method=median
batch: 0, error: 0.925 
batch: 100, error: 0.16500000000000004 
batch: 200, error: 0.08499999999999996 
done
Start nWorkers=20, beta=0, method=trimmedMean
batch: 0, error: 0.875 
batch: 100, error: 0.13 
bat

In [21]:
with open('data/ByzantineResults.pkl','rb') as fi:
    res = pk.load(fi)

In [22]:
res

{5: {('trimmedMean',
   0): {'training': ([0.905,
     0.92,
     0.88,
     0.905,
     0.9,
     0.835,
     0.855,
     0.84,
     0.795,
     0.825,
     0.75,
     0.645,
     0.725,
     0.6699999999999999,
     0.625,
     0.63,
     0.565,
     0.525,
     0.5349999999999999,
     0.5,
     0.44999999999999996,
     0.42000000000000004,
     0.355,
     0.35,
     0.32999999999999996,
     0.345,
     0.355,
     0.30000000000000004,
     0.365,
     0.265,
     0.20999999999999996,
     0.31999999999999995,
     0.22999999999999998,
     0.245,
     0.265,
     0.255,
     0.27,
     0.20999999999999996,
     0.20499999999999996,
     0.24,
     0.28500000000000003,
     0.22499999999999998,
     0.255,
     0.20999999999999996,
     0.18999999999999995,
     0.265,
     0.21499999999999997,
     0.17500000000000004,
     0.22499999999999998,
     0.18999999999999995],
    {'perBatch': [0.905,
      0.92,
      0.88,
      0.905,
      0.9,
      0.835,
      0.855,
      0.84